In [1]:
#
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import requests
import time
import pandas as pd

# Current News from NASA Website

In [2]:
#set up splinter browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

#visit url
url = "https://mars.nasa.gov/news/"
browser.visit(url)

#pull html text and parse
html_code = browser.html
soup = BeautifulSoup(html_code, "html.parser")

#grab needed info
news_title = soup.find('div', class_="bottom_gradient").text
news_p = soup.find('div', class_="rollover_description_inner").text

# Latest Featured Image

In [3]:
# Featured Image URL & visit
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

#navigate to link
browser.click_link_by_partial_text('FULL IMAGE')
#time.sleep(10)
# browser.click_link_by_partial_text('more info')

# #get html code once at page
# image_html = browser.html

# #parse
# soup = BeautifulSoup(image_html, "html.parser")

# #find path and make full path
# image_path = soup.find('figure', class_='lede').a['href']
# featured_image_url = "https://www.jpl.nasa.gov/" + image_path

In [4]:
browser.click_link_by_partial_text('more info')

#get html code once at page
image_html = browser.html

#parse
soup = BeautifulSoup(image_html, "html.parser")

#find path and make full path
image_path = soup.find('figure', class_='lede').a['href']
featured_image_url = "https://www.jpl.nasa.gov/" + image_path

# Mars Weather

In [92]:
marsweather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(marsweather_url)

weather_html = browser.html

soup = BeautifulSoup(weather_html, 'html.parser')

mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

# Mars Facts

In [93]:
#mars facts url and splinter visit
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

#get html
facts_html = browser.html

soup = BeautifulSoup(facts_html, 'html.parser')

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en-US" prefix="og: http://ogp.me/ns#" class="csstransforms csstransforms3d csstransitions has-cookie-bar cookie-bar-bottom-bar cookie-bar-bar"><head>\n<script src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.3LJqs_BmJws.O/m=auth/exm=plusone/rt=j/sv=1/d=1/ed=1/am=AQ/rs=AGLTcCON37lSPuqreC9udwNmc7WciA8O-A/cb=gapi.loaded_1" async=""></script><script src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.3LJqs_BmJws.O/m=plusone/rt=j/sv=1/d=1/ed=1/am=AQ/rs=AGLTcCON37lSPuqreC9udwNmc7WciA8O-A/cb=gapi.loaded_0" async=""></script><script src="https://pagead2.googlesyndication.com/pagead/js/r20171030/r20170110/osd.js"></script><script type="text/javascript" async="" src="https://apis.google.com/js/plusone.js" gapi_processed="true"></script><script src="https://pagead2.googlesyndication.com/pub-config/r20160913/ca-pub-4251889121233823.js"></script><script id="facebook-jssdk" src="//connect.facebook.net

In [94]:
#get the entire table
table_data = soup.find('table', class_="tablepress tablepress-id-mars")

In [95]:
#find all instances of table row
table_all = table_data.find_all('tr')

#set up lists to hold td elements which alternate between label and value
labels = []
values = []

#for each tr element append the first td element to labels and the second to values
for tr in table_all:
    td_elements = tr.find_all('td')
    labels.append(td_elements[0].text)
    values.append(td_elements[1].text)
        

In [96]:
#make a data frame and view
mars_facts_df = pd.DataFrame({
    "Label": labels,
    "Values": values
})

In [97]:
mars_facts_df

,Label,Values
0,Equatorial Diameter:,"6,792 km\n"
1,Polar Diameter:,"6,752 km\n"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)\n
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [98]:
# get html code for DataFrame
fact_table = mars_facts_df.to_html(header = False, index = False)
fact_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)\\n</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [101]:
# new url
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(usgs_url)

usgs_html = browser.html

soup = BeautifulSoup(usgs_html, "html.parser")

# gets class holding hemisphere picture
returns = soup.find('div', class_="collapsible results")
hemispheres = returns.find_all('a')

#setup list to hold dictionaries
hemisphere_image_urls =[]

for a in hemispheres:
    #get title and link from main page
    title = a.h3.text
    link = "https://astrogeology.usgs.gov" + a['href']
    
    #follow link from each page
    browser.visit(link)
    time.sleep(5)
    
    #get image links
    image_page = browser.html
    results = BeautifulSoup(image_page, 'html.parser')
    img_link = results.find('div', class_='downloads').find('li').a['href']
    
    # create image dictionary for each image and title
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = img_link
    
    hemisphere_image_urls.append(image_dict)
    
print(hemisphere_image_urls)
    
    #browser.visit


# usgs_html

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
